In [16]:
import pandas as pd
import numpy as np
import plotly.express as px

In [17]:
# paths corales puntacana

preds_path = r"C:\Users\mikej\Downloads\corales-puntacana-championship_course-adjustment-model.csv"

win_path = r"C:\Users\mikej\Downloads\pga_historical_outrights (22).csv"
top5_path = r"C:\Users\mikej\Downloads\pga_historical_outrights (21).csv"
top10_path = r"C:\Users\mikej\Downloads\pga_historical_outrights (20).csv"
top20_path = r"C:\Users\mikej\Downloads\pga_historical_outrights (19).csv"


In [18]:
def in_top20(a):
    if int(a) > 20:
        return "loss"
    return "win"

def in_top10(a):
    if int(a) > 10:
        return "loss"
    return "win"

def in_top5(a):
    if int(a) > 5:
        return "loss"
    return "win"

def in_win(a):
    if int(a) > 1:
        return "loss"
    return "win"

In [ ]:
# rbc heritage paths

    # read in datagolf
top20 = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (13).csv")
top10 = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (12).csv")
top5 = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (11).csv")
win = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (10).csv")

In [43]:
def get_dg_outcomes_table(preds_path,win_path,top5_path,top10_path,top20_path):

    # read in datagolf
    top20 = pd.read_csv(top20_path)
    top10 = pd.read_csv(top10_path)
    top5 = pd.read_csv(top5_path)
    win = pd.read_csv(win_path)
    
    # made dataframe of book odds
    odds = pd.concat([win,top5,top10,top20])
    odds = odds[['player_name','market','open_odds']].pivot(index='player_name',columns='market',values='open_odds').reset_index().round(2)
    odds.columns = ['player','odds_20','odds_10','odds_5','odds_win']
    
    # make dataframe of tournament outcomes (leaderboard)
    outcomes = top20[['player_name','outcome']]
    outcomes['outcome'] = outcomes['outcome'].str.strip("T")
    outcomes.columns = ['player','outcome']

    # make dataframe of datagolf predicted odds
    preds = pd.read_csv(preds_path)
    preds = preds[['player_name','top_20','top_10','top_5','win']].round(2)
    preds.columns = ['player','pred_20','pred_10','pred_5','pred_win']

    # merge books and datagolf
    df = pd.merge(odds,preds)[['player','odds_20','pred_20','odds_10','pred_10','odds_5','pred_5','odds_win','pred_win']]

    # add columns for difference in books odds and dg prediciton
    df['diff_20'] = (df['pred_20'] - df['odds_20']) / df['odds_20']
    df['diff_10'] = (df['pred_10'] - df['odds_10']) / df['odds_10']
    df['diff_5'] = (df['pred_5'] - df['odds_5']) / df['odds_5']
    df['diff_win'] = (df['pred_win'] - df['odds_win']) / df['odds_win']

    # merge in actual outcomes
    df = df.merge(outcomes,on='player')
    df = df.convert_dtypes()
    # df['outcome'] = df['outcome'].astype(int)

    # add outcome 'bools'
    # df['bool_20'] = df['outcome'].apply(in_top20)
    # df['bool_10'] = df['outcome'].apply(in_top10)
    # df['bool_5'] = df['outcome'].apply(in_top5)
    # df['bool_win'] = df['outcome'].apply(in_win)

    return df

    

    

    

    

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131 entries, 0 to 130
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   player    131 non-null    string 
 1   odds_20   131 non-null    Float64
 2   pred_20   131 non-null    Float64
 3   odds_10   131 non-null    Float64
 4   pred_10   131 non-null    Float64
 5   odds_5    131 non-null    Float64
 6   pred_5    131 non-null    Float64
 7   odds_win  131 non-null    Float64
 8   pred_win  131 non-null    Float64
 9   diff_20   129 non-null    Float64
 10  diff_10   131 non-null    Float64
 11  diff_5    116 non-null    Float64
 12  diff_win  84 non-null     Float64
 13  outcome   131 non-null    string 
dtypes: Float64(12), string(2)
memory usage: 16.9 KB


In [46]:
df = get_dg_outcomes_table(preds_path,win_path,top5_path,top10_path,top20_path)

C:\Users\mikej\AppData\Local\Temp\ipykernel_23568\1414055155.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
df.sort_values('odds_win',ascending=False)

,player,odds_20,pred_20,odds_10,pred_10,odds_5,pred_5,odds_win,pred_win,diff_20,diff_10,diff_5,diff_win,outcome
84,"Noren, Alex",0.38,0.52,0.58,0.35,0.24,0.22,0.08,0.06,0.368421,-0.396552,-0.083333,-0.250000,23
56,"Hojgaard, Nicolai",0.36,0.49,0.52,0.31,0.22,0.19,0.08,0.05,0.361111,-0.403846,-0.136364,-0.375000,CU
57,"Horschel, Billy",0.29,0.35,0.48,0.20,0.18,0.11,0.05,0.03,0.206897,-0.583333,-0.388889,-0.400000,1
95,"Rai, Aaron",0.28,0.41,0.45,0.25,0.15,0.14,0.04,0.04,0.464286,-0.444444,-0.066667,0.000000,CU
69,"Lashley, Nate",0.23,0.35,0.40,0.19,0.12,0.11,0.03,0.02,0.521739,-0.525000,-0.083333,-0.333333,CU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"Pumarol, Guillermo",0.01,0.00,0.02,0.00,0.00,0.00,0.00,0.00,-1.000000,-1.000000,NaN,NaN,CU
1,"Armour, Ryan",0.04,0.04,0.10,0.01,0.02,0.00,0.00,0.00,0.000000,-0.900000,-1.000000,NaN,67
99,"Santos, Julio",0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,NaN,-1.000000,NaN,NaN,CU
101,"Sekne, Herman Wibe",0.07,0.05,0.14,0.02,0.03,0.01,0.00,0.00,-0.285714,-0.857143,-0.666667,NaN,CU


In [34]:
px.bar(df.sort_values('odds_20')[-50:],
       x='player',
       y='diff_20')

In [ ]:
top20 = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (13).csv")
top10 = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (12).csv")
top5 = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (11).csv")
win = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (10).csv")

In [ ]:
top20['outcome'] = top20['outcome'].str.strip("T")

outcomes = top20[['player_name','outcome']]
outcomes.columns = ['player','outcome']

In [ ]:
outcomes#.head(2)

In [ ]:
odds = pd.concat([win,top5,top10,top20])

odds = odds[['player_name','market','open_odds']].pivot(index='player_name',columns='market',values='open_odds').reset_index().round(2)

odds.columns = ['player','odds_20','odds_10','odds_5','odds_win']

odds = odds[['player','odds_20','odds_10','odds_5','odds_win']]

odds#.head(2)

In [ ]:
preds = pd.read_csv(r"C:\Users\mikej\Downloads\rbc-heritage_course-adjustment-model.csv")

preds = preds[['player_name','top_20','top_10','top_5','win']].round(2)

preds.columns = ['player','pred_20','pred_10','pred_5','pred_win']

preds#.head(2)

In [ ]:
df = pd.merge(odds,preds)[['player','odds_20','pred_20','odds_10','pred_10','odds_5','pred_5','odds_win','pred_win']]

In [ ]:
df#.head(2)

In [ ]:
df['diff_20'] = df['pred_20'] - df['odds_20']
df['diff_10'] = df['pred_10'] - df['odds_10']
df['diff_5'] = df['pred_5'] - df['odds_5']
df['diff_win'] = df['pred_win'] - df['odds_win']

In [ ]:
df

In [ ]:
df = df.merge(outcomes,on='player')
df

In [ ]:
def in_top20(a):
    if a > 20:
        return "loss"
    return "win"

def in_top10(a):
    if a > 10:
        return "loss"
    return "win"

def in_top5(a):
    if a > 5:
        return "loss"
    return "win"

def in_win(a):
    if a > 1:
        return "loss"
    return "win"

In [ ]:
df['outcome'] = df['outcome'].astype(int)

df['outcome_bool_20'] = df['outcome'].apply(in_top20)
df['outcome_bool_10'] = df['outcome'].apply(in_top10)
df['outcome_bool_5'] = df['outcome'].apply(in_top5)
df['outcome_bool_win'] = df['outcome'].apply(in_win)

In [ ]:
df

In [ ]:
top20s = df[['player','outcome','odds_20','pred_20','diff_20','outcome_bool_20']].sort_values('odds_20',ascending=False)
# top10s = df[['player','odds_10','pred_10','diff_10','outcome']].sort_values('odds_10',ascending=False)
# top5s = df[['player','odds_5','pred_5','diff_5','outcome']].sort_values('odds_5',ascending=False)
# wins = df[['player','odds_win','pred_win','diff_win','outcome']].sort_values('odds_win',ascending=False)
top20s[:20]

In [ ]:

px.scatter(df,
           x='odds_20',
           y='pred_20',
           hover_name='player',
           color='outcome_bool_20',
           color_discrete_sequence=px.colors.qualitative.Safe,
           trendline = 'ols',trendline_scope='overall',trendline_color_override = 'darkslategrey',
           width=800,
           height=700,
           size='diff_20')#.add_shape(type="line", x0=0, y0=0, x1=1, y1=1)

In [ ]:

px.scatter(df,
           x='diff_10',
           y='pred_10',
           hover_name='player',
           color='outcome_bool_10',
           color_discrete_sequence=px.colors.qualitative.Safe,
           trendline = 'ols',trendline_scope='overall',trendline_color_override = 'darkslategrey',
           width=800,
           height=700,
           size='odds_10')#.add_shape(type="line", x0=0, y0=0, x1=1, y1=1)

In [ ]:


px.scatter(df,
           x='diff_5',
           y='pred_5',
           hover_name='player',
           color='outcome_bool_5',
           color_discrete_sequence=px.colors.qualitative.Safe,
           trendline = 'ols',trendline_scope='overall',trendline_color_override = 'darkslategrey',
           width=800,
           height=700,
           size='odds_5')#.add_shape(type="line", x0=0, y0=0, x1=1, y1=1)

In [ ]:


# top5s['outcome'] = top5s['outcome'].astype('int')

px.scatter(df,
           x='diff_win',
           y='pred_win',
           hover_name='player',
           color='outcome_bool_win',
           color_discrete_sequence=px.colors.qualitative.Safe,
           trendline = 'ols',trendline_scope='overall',trendline_color_override = 'black',
           width=800,
           height=700,)

In [ ]:
temp = df.sort_values('outcome')

In [ ]:
px.bar(temp.sort_values('outcome'),#,ascending=False),
    x='player',
    y='outcome',
    color='diff_20').add_vline(x='Kim, Si Woo')

In [ ]:
df

In [ ]:
px.scatter(df,
           x='odds_20',
           y='pred_20',
           color='outcome_bool_20',
           trendline='ols',trendline_scope='overall')

In [ ]:
df.diff_20.hist()